In [1]:
import os
from typing import Tuple
import numpy as np
import torch
from torch.utils.data import Dataset
from torchvision.io.image import read_image
from torchvision.transforms import v2
import matplotlib.pyplot as plt
from data.utils import walk_path

In [2]:
from utils.helper_functions import StreetHazardsDataModule
from data.shift_dataset import LabelFilter

In [3]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor, TQDMProgressBar
from nets.wrapper import Wrapper

In [4]:
model = Wrapper("resnet50", 13, .0001)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home/mbikandi/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth
100%|██████████| 97.8M/97.8M [00:14<00:00, 6.86MB/s]


In [12]:
sum(x.nelement() for x in model.model.classifier.parameters())

16251757

In [17]:
with torch.no_grad():
    model.model(torch.randn((1, 3, 1260, 800), dtype=torch.float32, device="cuda"))

In [ ]:
ckpt = ModelCheckpoint(
        monitor="val_miou",
        mode="max",
        save_top_k=2,
        filename='{epoch}-{step}-{val_miou:.4f}'
    )

In [ ]:
tr = Trainer(default_root_dir="./test", accelerator="cuda", callbacks=[ckpt, LearningRateMonitor(logging_interval="epoch"),
        TQDMProgressBar(refresh_rate=5)], max_epochs=2)

In [ ]:
tr.fit(model=model, datamodule=dm)